In [16]:
!ls
!rm -rf jpeg
%cd /kaggle/working/


datasets  dog.jpeg  runs  yolo12n.pt
/kaggle/working


In [17]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [4]:
!pip install -U ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.4/8062.4 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

In [6]:
# Run inference on an image with YOLO12n
# !yolo predict model=yolo12n.pt source='https://ultralytics.com/images/zidane.jpg'

%cd {HOME}
!yolo predict mode=predict model=yolo12n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

/kaggle/working
Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,590,824 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /kaggle/working/dog.jpeg: 640x384 1 person, 1 car, 2 dogs, 70.6ms
Speed: 19.5ms preprocess, 70.6ms inference, 473.9ms postprocess per image at shape (1, 3, 640, 384)
Results saved to /kaggle/working/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
%cd {HOME}
# Image('runs/detect/predict/dog.jpeg')
Image(filename='runs/detect/predict/dog.jpg', height=600)

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="n12mRmQdWmrjCSKFgJMC")
project = rf.workspace("waste-detections-by-fu").project("waste-detections-c0mkz")
version = project.version(6)
dataset = version.download("yolov12")

In [9]:
dataset.location

'/kaggle/working/datasets/Waste-Detections-6'

In [10]:
!pip install Pillow


In [ ]:
# Train YOLO12n
%cd {HOME}

!yolo train model=yolo12n.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 plots=True device=0,1

In [22]:
!ls {HOME}/runs/detect/train/

args.yaml			 results.csv	       val_batch0_labels.jpg
BoxF1_curve.png			 results.png	       val_batch0_pred.jpg
BoxP_curve.png			 train_batch0.jpg      val_batch1_labels.jpg
BoxPR_curve.png			 train_batch16000.jpg  val_batch1_pred.jpg
BoxR_curve.png			 train_batch16001.jpg  val_batch2_labels.jpg
confusion_matrix_normalized.png  train_batch16002.jpg  val_batch2_pred.jpg
confusion_matrix.png		 train_batch1.jpg      weights
labels.jpg			 train_batch2.jpg


In [ ]:
%cd {HOME}
Image.open(f'{HOME}/runs/detect/train/confusion_matrix.png')

In [ ]:
%cd {HOME}
Image.open(f'{HOME}/runs/detect/train/results.png')

In [ ]:
%cd {HOME}
Image.open(f'{HOME}/runs/detect/train/val_batch0_pred.jpg')

In [28]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

/kaggle/working
Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2187.7±907.7 MB/s, size: 80.2 KB)
val: Scanning /kaggle/working/datasets/Waste-Detections-6/valid/labels.cache... 427 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 427/427 8.5Mit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 27/27 5.2it/s 5.2s0.1s
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
                   all        427       1050      0.695      0.611      0.658      0.539
                 Paper        151        275      0.736      0.585      0.698      0.5

In [29]:
!yolo export model={HOME}/runs/detect/train/weights/best.pt format=torchscript

Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.3 MB)

TorchScript: starting export with torch 2.6.0+cu124...
TorchScript: export success ✅ 3.2s, saved as '/kaggle/working/runs/detect/train/weights/best.torchscript' (10.4 MB)

Export complete (3.9s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train/weights/best.torchscript imgsz=640 data=/kaggle/working/datasets/Waste-Detectio

In [30]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

/kaggle/working
Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs

image 1/283 /kaggle/working/datasets/Waste-Detections-6/test/images/20250303_141402_jpg.rf.eca6609b65e2645932230f9186be83f4.jpg: 640x640 8 Plastic Bottles, 15.9ms
image 2/283 /kaggle/working/datasets/Waste-Detections-6/test/images/20250303_141446_jpg.rf.fff8ddd58bfa68db3ad17b97c8d21949.jpg: 640x640 1 Paper, 6 Plastic wastes, 14.3ms
image 3/283 /kaggle/working/datasets/Waste-Detections-6/test/images/20250303_141649_jpg.rf.30d1a279399f53b8d35f8567c01d9874.jpg: 640x640 4 Plastic wastes, 13.4ms
image 4/283 /kaggle/working/datasets/Waste-Detections-6/test/images/20250303_141652_jpg.rf.f9c3ff2eb28a69c88222dc0a89e4a0ac.jpg: 640x640 4 Plastic wastes, 13.3ms
image 5/283 /kaggle/working/datasets/Waste-Detections-6/test/images/20250303_141654_jpg.rf.0a76417ecdcfc7ae4fffb7fa0cdb3d21.jpg: 640x640 7 Plastic wastes, 13.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
! zip -r yolov12n_recyclablewaste.zip {HOME}/runs/